In [1]:
#!pip install bertopic

In [16]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from pathlib import Path
import pandas as pd
import regex

In [17]:
def read_seed(seed_file):
    """
    Construct topic list from seed file (.md format)
    """
    topics = []
    pattern = regex.compile("^\[(\d+)\] ([\w\s]+) \(Count: (\d+)\): (.+)")
    hierarchy = open(seed_file, "r").readlines()
    for res in hierarchy:
        res = res.strip().split("\n")
        for r in res:
            #Remove characters before first whitespace " "
            r = r.strip().split(" ")[1:]
            if r != []:
                topics.append(r)
    return topics

In [32]:
input_file_path = "../../topic-gpt-fork/data/input/uniform_sample.jsonl"
# seed_topic_file_path = "../../topic-gpt-fork/prompt/bertopic_comparison/seed_out.md"
seed_topic_file_path = "../../topic-gpt-fork/prompt/bertopic_comparison/seed_in.md"
# outputh_path = "../../topic-gpt-fork/data/output/llama-3-70b-chat/bertopic_comparison/bertopic_output_out.csv"
outputh_path = "../../topic-gpt-fork/data/output/llama-3-70b-chat/bertopic_comparison/bertopic_output_in.csv"

In [33]:

df = pd.read_json(str(input_file_path), lines=True)
docs = df["text"].tolist()

seed_topics = read_seed(seed_topic_file_path)

In [36]:


representation_model = KeyBERTInspired()

topic_model = BERTopic(seed_topic_list=seed_topics, representation_model=representation_model, verbose=True)


In [37]:
topics, probs = topic_model.fit_transform(docs)

2024-05-12 14:28:47,891 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 32/32 [00:09<00:00,  3.36it/s]
2024-05-12 14:28:57,495 - BERTopic - Embedding - Completed ✓
2024-05-12 14:28:57,495 - BERTopic - Guided - Find embeddings highly related to seeded topics.
Batches: 100%|██████████| 1/1 [00:00<00:00, 144.97it/s]
2024-05-12 14:28:57,507 - BERTopic - Guided - Completed ✓
2024-05-12 14:28:57,508 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-12 14:28:59,190 - BERTopic - Dimensionality - Completed ✓
2024-05-12 14:28:59,190 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-12 14:28:59,204 - BERTopic - Cluster - Completed ✓
2024-05-12 14:28:59,205 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-12 14:29:00,795 - BERTopic - Representation - Completed ✓


In [38]:
#Store the resulting names of topics from Pandas DF in a file
topic_model.get_topic_info().to_csv(outputh_path, index=False)
